In [23]:
#Import our libraries
import time
import requests
from requests.auth import HTTPBasicAuth

import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy
import utm_helper_functions as hf

In [16]:
#Set the date range variables for the date range we want pulled
year = '2023'
month = '03'

In [17]:
#Bring in the timesheet endpoint to get the pay period
url="https://kN21.ultipro.com/UtmOdataServices/api/Timesheet?$filter=year(Starts) eq " + year + " and month(Starts) eq " + month

#Our username and password for the API authentication
username = "znarkin"
password = "v#2OT*aOm#JC/xq+MC_}"


#The response variable
response = requests.get(url, auth=HTTPBasicAuth(username, password))

In [18]:
data = response.json()
#data

In [19]:
df = pd.DataFrame(data['value'])
df

,Id,EmployeeId,Starts,Ends,MissPunch,Tardy,SchHr,RegHr,Ot1,Ot2,...,NoMeal,Overtime,ShortWkDay,AppPunch,AuthOt,PaidBrk,MlPremCnt,MlPremWav,BkPremCnt,BkPremWav
0,523596,1289,2023-03-12T00:00:00-05:00,2023-03-25T00:00:00-04:00,0,0,0.0,80.0001,4.9166,0.0,...,3,4.92,0,19.0,0.0,0,0,0,0,0
1,524132,5432,2023-03-12T00:00:00-05:00,2023-03-25T00:00:00-04:00,0,0,0.0,80.0000,4.6332,0.0,...,1,4.63,0,20.0,0.0,0,0,0,0,0
2,525700,246,2023-03-12T00:00:00-05:00,2023-03-25T00:00:00-04:00,0,0,0.0,66.5335,0.0000,0.0,...,0,0.00,0,17.0,0.0,0,0,0,0,0
3,527699,2288,2023-03-12T00:00:00-05:00,2023-03-25T00:00:00-04:00,0,0,0.0,80.0000,1.6168,0.0,...,0,1.62,0,15.0,0.0,0,0,0,0,0
4,527805,6581,2023-03-12T00:00:00-05:00,2023-03-25T00:00:00-04:00,0,0,0.0,0.0000,0.0000,0.0,...,0,0.00,0,1.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,559159,7691,2023-03-26T00:00:00-04:00,2023-04-08T00:00:00-04:00,0,0,0.0,0.0000,0.0000,0.0,...,0,NaN,0,NaN,NaN,0,0,0,0,0
4180,559183,7684,2023-03-26T00:00:00-04:00,2023-04-08T00:00:00-04:00,0,0,0.0,0.0000,0.0000,0.0,...,0,0.00,0,0.0,0.0,0,0,0,0,0
4181,559186,7705,2023-03-26T00:00:00-04:00,2023-04-08T00:00:00-04:00,0,0,0.0,0.0000,0.0000,0.0,...,0,0.00,0,0.0,0.0,0,0,0,0,0
4182,559263,7682,2023-03-26T00:00:00-04:00,2023-04-08T00:00:00-04:00,0,0,0.0,0.0000,0.0000,0.0,...,0,0.00,0,0.0,0.0,0,0,0,0,0


In [20]:
df.columns

Index(['Id', 'EmployeeId', 'Starts', 'Ends', 'MissPunch', 'Tardy', 'SchHr',
       'RegHr', 'Ot1', 'Ot2', 'Ot3', 'Ot4', 'Ot5', 'PayAmt', 'Submit',
       'Approve', 'SysUpdate', 'EarlyIn', 'GraceIn', 'EarlyOut', 'LateOut',
       'LongMeal', 'ShortMeal', 'LongBreak', 'ShortBreak', 'Absences',
       'DayApprove', 'Status', 'AutoMeal', 'EarlyOutG', 'NoMeal', 'Overtime',
       'ShortWkDay', 'AppPunch', 'AuthOt', 'PaidBrk', 'MlPremCnt', 'MlPremWav',
       'BkPremCnt', 'BkPremWav'],
      dtype='object')

#Get list of columns formatted to create SQL table
time_columns_cleaned = []
for each in list(df.columns):
    cleaned_column = each + ' ' + 'NVARCHAR(255)' 
    time_columns_cleaned.append(cleaned_column)

str(time_columns_cleaned).replace('\'', '').replace('[', '').replace(']', '')

In [21]:
#Export data as CSV
df.to_csv('ukg_bulk_insert.csv', index=False)

In [24]:
#Pull data into SQL
hf.create_sql_table_v2('ZZ_TEST_UTM_TIMESHEET')